In [1]:
# default_exp convert_time_log

# convert time log

> Convert 'client' time log to trace file format for visualization

In [2]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

In [3]:
#export
import xml.etree.ElementTree as ET

import tempfile
import sys

import numbers
import decimal

class trace_log():
    def __init__(self,fp_output):
        self.outfile = fp_output
        self.outfile.write('{ "traceEvents": \n')
        self.outfile.write('[')
        
    def write_element(self, event_type, name, categories, pid,tid, ts,additional_args=None):
        ''' {"name": "Asub", "cat": "PERF", "ph": "B", "pid": 22630, "tid": 22630, "ts": 829}'''
        
        line = '"name": "%s", "cat": %s, "ph": "%s", "pid": %s, "tid": %s, "ts": %s,' %(name,categories,event_type,pid,tid,ts)
        if additional_args is not None and len(additional_args) > 0:
            line += format_value('args',additional_args) + ','
        self.outfile.write('{'+line+'},\n')

    def write_start_element(self, name, categories, pid,tid, ts,additional_args=None):
        self.write_element('B',name,categories,pid,tid,ts,additional_args)
        
    def write_end_element(self, name, categories, pid,tid, ts):
        self.write_element('E',name,categories,pid,tid,ts)
     
    
    def close(self):
        self.outfile.write('],\n')
        self.outfile.write(''' "displayTimeUnit": "ms",
 "systemTraceEvents": "SystemTraceData",
 "otherData": { "version": "PowerOn Client Trace"},
}
''')
        self.outfile.close()


def format_value(name, value):
    ''' takes name and value and returns a string for the value element'''
    formatted_string = ''
    if name is not None:
        formatted_string += '"%s": '%name
    if isinstance(value, numbers.Number):
        formatted_string += '%s'%value
    elif isinstance(value, dict):
        formatted_string +=  '{'
        sep = ''
        for key1,value1 in value.items():
            formatted_string += sep + format_value(key1,value1) 
            sep = ','
        formatted_string += '}'
    elif isinstance(value, list):
        formatted_string += '['
        sep = ''
        for item in value:
            formatted_string += sep + format_value(None,item) 
            sep = ','
        formatted_string += ']'
    else:
        formatted_string += '"%s"' %value
    
    return formatted_string 


In [4]:
#hide
a = {'a':1,'b':2,'c':{'d':3,'e':[{'name':'hello','value':'world'},{'name':'hello2','value':'world2'}]}}
format_value('args',a)

test_eq(format_value('args',a),'"args": {"a": 1,"b": 2,"c": {"d": 3,"e": [{"name": "hello","value": "world"},{"name": "hello2","value": "world2"}]}}')

In [5]:
#export

def strip_extra_EnmacClientTime_elements(filename,fp):
    ''' since we may have multiple EnmacClientTiming tags in the document we need to strip them out and add one at the end'''
    
    fp.write("<EnmacClientTiming>\n")

    if sys.version_info[0] < 3:
        file = open(filename, 'r')
    else:
        file = open(filename, 'r', encoding='utf8')

    if file is None:
        print('Error opening file: %s' %filename)
        return
    for line in file:
        l = line.rstrip()
        if l == '<EnmacClientTiming>' or l == '</EnmacClientTiming>':
            continue
        fp.write(line)
         
    fp.write("</EnmacClientTiming>\n")



In [6]:
#export

import datetime 

if sys.version_info[0] < 3:
    epoch = datetime.datetime(1970, 1, 1)
else:
    epoch = datetime.datetime(1970, 1, 1, tzinfo=datetime.timezone.utc)

#epoch = datetime.datetime.fromtimestamp(0,datetime.timezone.utc) # 

def unix_time_millis(dt):
    return (dt - epoch).total_seconds() * 1000.0

def parse_dt(time_str):
    ''' Parse string in format 2021-12-07T08:51:46.479299+00:00  
    return datetime'''

    if sys.version_info[0] < 3:
        time_str = time_str[:-6]
        return datetime.datetime.strptime(time_str, '%Y-%m-%dT%H:%M:%S.%f')
    else:
        return datetime.datetime.strptime(time_str, '%Y-%m-%dT%H:%M:%S.%f%z').replace(tzinfo=datetime.timezone.utc)

    return datetime.datetime.strptime(time_str, "%Y-%m-%dT%H:%M:%S.%f%z")

def parse_dt_to_millis(time_str):
    return unix_time_millis(parse_dt(time_str))

parse_dt_to_millis('2021-12-07T08:51:46.479299+00:00')


1638867106479.299

In [7]:
#export

class parse_timing_log():
    
    def __init__(self, fp_input, fp_output):
        
        self.parse_functions = {
        'Timing' : self.skip_element,
        'Parameters' : self.skip_element,
        'DataTables' : self.skip_element,
        'HttpWebRequest' : self.parse_HttpWebRequest,
        'CreateDataSet' : self.parse_CreateDataSet,
        'FormAction' : self.parse_FormAction,
        
        }
        
        self.fp = fp_input
        self.fp_output = fp_output
        #self.elements = []
        #self.path_list = []
        self.level = 0
        self.t = trace_log(fp_output)
        
        self.parse2()

    def skip_element(self, elment):
        return 0


    def parse_datatables_element(self, elem):
        ''' Parse element datatables which looks like:
        <DataTables name="AvailableResources" tables="5" error="false">
            <DataTable name="AVAILABLERESOURCES" rows="7"/>
            <DataTable name="CONTACTDETAILS" rows="7"/>
            <DataTable name="RESOURCEZONES" rows="0"/>
            <DataTable name="ASSIGNED_INCIDENTS" rows="0"/>
            <DataTable name="RESOURCE_OZ_ASSOCIATIONS" rows="2"/>
        </DataTables>
    return a list of date tables as arguments...'''
        datatables  = { 'name': elem.attrib['name'], 'number_of_tables': elem.attrib['tables'], 'error': elem.attrib['error'] }

        tables = []
        for child in elem:
            tables.append({ 'name':child.attrib['name'], 'rows':child.attrib['rows'] })
        datatables['tables'] = tables
        return datatables

    def parse_parameter_element(self, element):
        ''' Parse element parameter which looks like:
        <Parameters postSize="105">
            <Parameter name="searchlon" value="28.020557"/>
            <Parameter name="positionupdatewithinmins" value="30"/>
            <Parameter name="radiusinmiles" value="False"/>
            <Parameter name="searchradius" value="10"/>
            <Parameter name="searchlat" value="-26.033414"/>
        </Parameters>
        returns a list of parameters as arguments...'''
        parameters = {'postSize': element.attrib['postSize']}
        params = {}
        for child in element:
            params[child.attrib['name']] = child.attrib['value']  
            #params.append({'name': child.attrib['name'], 'value': child.attrib['value']})
        parameters['params'] = params
        return parameters
        
    def get_end_time(self, element):
        ''' gets the timing element in element and returns the ms'''
        timing = element.find('Timing')
        if timing is None:
            print('no timing element found')
            return None
        ts = timing.attrib["end"]
        ms = parse_dt_to_millis(ts)

        if timing.attrib["thread"] != element.attrib["thread"]:
            print('missmatched timing threads ??')
        return ms

    def parse_HttpWebRequest(self,elem):
        ''' Parse element HttpWebRequest which looks like:

        <HttpWebRequest start="2021-12-07T17:46:43.910299+00:00" client="hydrogen-w11" thread="282" method="POST" uri="https://trn1nms/enmac/resources/available" status="OK" serverElapsed="0.33" serverCPU="0.0257">
        <Parameters postSize="105">
            <Parameter name="searchlon" value="28.020557"/>
            <Parameter name="positionupdatewithinmins" value="30"/>
            <Parameter name="radiusinmiles" value="False"/>
            <Parameter name="searchradius" value="10"/>
            <Parameter name="searchlat" value="-26.033414"/>
        </Parameters>
        <Timing thread="282" end="2021-12-07T17:46:45.688867+00:00" elapsed="1.78"/>
        </HttpWebRequest>
        
        '''
        name = elem.attrib["uri"]
        categories = '"HttpWebRequest"'
        pid = 1
        tid = elem.attrib["thread"]
        ts = elem.attrib["start"]
        ms = parse_dt_to_millis(ts)

        parameters = elem.find('Parameters')
        p = self.parse_parameter_element(parameters)
        additional_args = { 
                    'parameters': p,
                    'start':ts,
                    'client':elem.attrib["client"], 
                    'method':elem.attrib["method"],
                    'status':elem.attrib["status"],
                    'serverElapsed': float(elem.attrib["serverElapsed"]),
                    'serverCPU': float(elem.attrib["serverCPU"]) }
        
        
        self.t.write_start_element(name,categories ,pid,tid,ms,additional_args)
         
        self.parse_elements(elem)
        ms = self.get_end_time(elem)
        self.t.write_end_element(name,categories ,pid,tid,ms)
        return 0
       
        
    def parse_CreateDataSet(self,elem):
        '''
        Parse element CreateDataSet which looks like:
        <CreateDataSet start="2021-12-07T17:50:36.520401+00:00" client="hydrogen-w11" thread="273" path="/enmac/swex/outages">
        ....
        <DataTables name="AvailableResources" tables="5" error="false">
        <DataTable name="AVAILABLERESOURCES" rows="7"/>
        <DataTable name="CONTACTDETAILS" rows="7"/>
        <DataTable name="RESOURCEZONES" rows="0"/>
        <DataTable name="ASSIGNED_INCIDENTS" rows="0"/>
        <DataTable name="RESOURCE_OZ_ASSOCIATIONS" rows="2"/>
        </DataTables>
         <Timing thread="282" end="2021-12-07T17:46:45.688867+00:00" elapsed="1.78"/>
        </CreateDataSet>
        '''
        name = elem.attrib["path"]
        categories = '"CreateDataSet"'
        pid = 1
        tid = elem.attrib["thread"]
        ts = elem.attrib["start"]
        ms = parse_dt_to_millis(ts)

        datatables = elem.find('DataTables')
        dt = self.parse_datatables_element(datatables)

        additional_args = { 
                    'datatables': dt,
                    'start':ts,
                    'client':elem.attrib["client"]}
        self.t.write_start_element(name,categories ,pid,tid,ms,additional_args)

        self.parse_elements(elem)
    
        ms = self.get_end_time(elem)
        self.t.write_end_element(name,categories ,pid,tid,ms)
        return 0 
        
    def parse_FormAction(self,elem):
        '''
        <FormAction start="2021-12-07T17:46:23.903476+00:00" client="hydrogen-w11" thread="18" 
        form="MainForm" action="SendNetServerMessage(&#39;MD_SET_PIPE_MESSAGE_FILTER&#39;, &#39;135&#39;)">>
        '''
        name = elem.attrib["action"]
        form = elem.attrib["form"]
        categories = '"FormAction,%s"'%form
        pid = 1
        tid = elem.attrib["thread"]
        ts = elem.attrib["start"]
        ms = parse_dt_to_millis(ts)

        additional_args = { 
                    'start':ts,
                    'client':elem.attrib["client"],
                    'form': form}
        self.t.write_start_element(name,categories ,pid,tid,ms,additional_args)


        self.parse_elements(elem)
        ms = self.get_end_time(elem)
        self.t.write_end_element(name,categories ,pid,tid,ms)

        return 0
        
    def parse_elements(self, elem):
        for child in elem:
            func= self.parse_functions.get(child.tag,lambda :-1)
            if func(child) <0:
                print('Unknown tag: elem.tag')
            
    def parse2(self):
        tree = ET.parse(self.fp)
        root = tree.getroot()
        self.parse_elements(root)
        self.t.close()
            
            

In [11]:
#hide
fp = open('logs/very_short_log.xml','r')
fp_output = open('logs/very_short_log.json','w')
parse_timing_log(fp,fp_output)

In [12]:
#export
try: from nbdev.imports import IN_NOTEBOOK
except: IN_NOTEBOOK=False
    
if __name__ == "__main__" and not IN_NOTEBOOK:
    import argparse
    import os
    import sys
    import random

    ap = argparse.ArgumentParser()
    ap.add_argument('-p', '--nopreprocess', required=False,
                    help="don't preprocess the file to strip out extra EnmacClientTiming tags in the file", default=True, action='store_false')
    ap.add_argument("-f", "--file", required=True, help="file to parse to generate timing log file from ")
    ap.add_argument("-o", "--output", required=False, help="output file name",default = '')
    ap.add_argument("-s","--stdout", required=False, help="print to stdout",default = False, action='store_true')

    args = vars(ap.parse_args())

    tmp_file = ''

    if args['nopreprocess']:
        tmp_file = '/tmp/cronos_' + str(random.randint(1, 1000000)) + '.tmp'
        fp = open(tmp_file, 'w+')
        strip_extra_EnmacClientTime_elements(args['file'],fp)
        fp.close()
        fp = open(tmp_file, 'r')
    else:
        fp = open(args['file'])
    

    if args['stdout']:
        fo = sys.stdout
        parse_timing_log(fp,sys.stdout)
    elif args['output'] == '':
        outfilename =  os.path.splitext(args['file'])[0] + '.json'
        fo = open(outfilename,'w')
    else:
        outfilename = args['output']
        fo = open(outfilename,'w')
    

    parse_timing_log(fp,fo)
    fo.close()
    fp.close()

    if tmp_file != '':
        os.remove(tmp_file)


In [13]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 01_convert_time_log.ipynb.
Converted index.ipynb.
